![alt text](./Cerny_logo_1.jpg)

# Analysis of Cerny ventilation recordings

#### Analysis of ventilator data obtained from ELBW (birth weight <1000 g) infants transferred ex utero to receive level 3 neonatal 

##### intensive care during the first 24 hours of life among recordings `AL000001 - AL001100` and  `AT000001 - AT000818`

#### Author: Dr Gusztav Belteki

### 1. Import the required libraries and set options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt

import os
import sys
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline
matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None) 

In [ ]:
print('Python version: {}'.format(sys.version))
print('pandas version: {}'.format(pd.__version__))
print('matplotlib version: {}'.format(matplotlib.__version__))
print('NumPy version: {}'.format(np.__version__))
print('SciPy version: {}'.format(sp.__version__))
print('IPython version: {}'.format(IPython.__version__))

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'ELBW'

# Path to project folder containing clinical data (current weights only) and for export of results
PATH = os.path.join(os.sep, 'Users', 'guszti', 'Library', 'Mobile Documents', 'com~apple~CloudDocs', 
                            'Documents', 'Research', 'Ventilation')

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Folder containing the file with the manually collected clinical data
DIR_READ_CLIN = os.path.join(os.sep, PATH, 'ventilation_fabian_combined')

# Data loaded from both directories
DIR_READ_1 = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian')
DIR_READ_2 = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian_new')

# Results will be written in this folder
DIR_WRITE =  os.path.join(os.sep, PATH, 'ventilation_fabian_combined', 'Analyses', TOPIC)
os.makedirs(DIR_WRITE, exist_ok=True)

# Images and raw data will be written on an external hard drive
DATA_DUMP = os.path.join(os.sep, 'Volumes', DRIVE, 'data_dump', 'fabian_combined', TOPIC)
os.makedirs(DATA_DUMP, exist_ok=True)

In [ ]:
DIR_READ_CLIN, DIR_READ_1, DIR_READ_2, DIR_WRITE, DATA_DUMP

### 3. Import computationally retrieved data

#### A. Clinical data

These data have been computationally generated from scanned transport records

In [ ]:
with open(os.path.join(DIR_READ_1, 'clin_df_1_1100.pickle'), 'rb') as handle:
    clin_df_old = pickle.load(handle)
with open(os.path.join(DIR_READ_2, 'clin_df_new.pickle'), 'rb') as handle:
    clin_df_new = pickle.load(handle)

clin_data_comput = pd.concat([clin_df_old, clin_df_new])

clin_data_comput['Birth Weight'] = clin_data_comput['Birth Weight'].astype('float')

In [ ]:
clin_data_comput.info()

#### B. Blood gases

Blood gases have been pre-processed by other Jupyter Notebooks. They have also been reviewed and outlier values manually verified.

In [ ]:
with open(os.path.join(DIR_READ_1, 'blood_gases_1_1100.pickle'), 'rb') as handle:
    blood_gases_old = pickle.load(handle)
with open(os.path.join(DIR_READ_2, 'blood_gases_new.pickle'), 'rb') as handle:
    blood_gases_new = pickle.load(handle)

blood_gases_comput = {**blood_gases_old, **blood_gases_new,}
len(blood_gases_comput)

#### C. Ventilation modes

Table with ventilation modes has been generated by other Jupyter Notebooks. They only contain cases when respiratory support on the Fabian was longer than 10 minutes.

In [ ]:
with open(os.path.join(DIR_READ_1, 'vent_modes_ventilated_1_1100_plus.pickle'), 'rb') as handle:
    vent_modes_ventilated_1_1100 = pickle.load(handle)
with open(os.path.join(DIR_READ_1, 'vent_modes_noninvasive_1_1100_plus.pickle'), 'rb') as handle:
    vent_modes_noninvasive_1_1100 = pickle.load(handle)
    
with open(os.path.join(DIR_READ_2, 'vent_modes_trimmed_new_1_1305.pickle'), 'rb') as handle:
    vent_modes_trimmed_new_1_1305 = pickle.load(handle)

# Rename a column which is named differently than in the other table
vent_modes_trimmed_new_1_1305.rename({'VG':'VG_on'}, axis=1, inplace=True)

vent_modes = pd.concat([vent_modes_ventilated_1_1100, vent_modes_noninvasive_1_1100, vent_modes_trimmed_new_1_1305])

# Keep only relevant columns present in all tables
columns_to_keep = ['IPPV', 'PSV', 'SIMV', 'SIMVPSV', 'SIPPV', 'O2therapy', 'DUOPAP', 'NCPAP',
       'CPAP', 'HFO', 'VG_on', 'total', ]
vent_modes = vent_modes[columns_to_keep]
len(vent_modes)

In [ ]:
vent_modes.head()

### 4. Limit patients to those who meet the selection criteria

#### A. Limit analysis to cases not later than `AT000818`

In [ ]:
new_index = [item for item in clin_data_comput.index if item.startswith('AL') or int(item[2:].lstrip('0')) <= 818]
clin_data_comput = clin_data_comput.loc[new_index]
len(clin_data_comput)

#### B. Only include patients who are born with <1000 g birth weight

In [ ]:
clin_data_comput_ELBW = clin_data_comput[clin_data_comput['Birth Weight'] < 1000]
len(clin_data_comput_ELBW)

In [ ]:
# Exclude two infants whose birth weight was actually < 1000 g after detailed review of clinical notes
clin_data_comput_ELBW = clin_data_comput_ELBW.drop(['AL000003', 'AL000400'])
len(clin_data_comput_ELBW)

#### C. Exclude infants who were not transferred on the first day of life

`Postnatal Age` is the chronological age at the end of the recording (transfer)

In [ ]:
clin_data_comput_ELBW = clin_data_comput_ELBW[clin_data_comput_ELBW['Postnatal Age'] < pd.to_timedelta('1D')]
len(clin_data_comput_ELBW)

#### D. Exclude one infant who was transferred from one level 3 NICU to another

- due to capacity reasons (AL000829) 
- as iNO was not available on the first NICU (AT000707).

In [ ]:
clin_data_comput_ELBW = clin_data_comput_ELBW.drop(['AL000829', 'AT000707'])
len(clin_data_comput_ELBW)

#### E. Exclude infants with major congenital malformations 

- 'AL000084': oesophagus atresia
- 'AL000204': gastroschisis

In [ ]:
clin_data_comput_ELBW = clin_data_comput_ELBW.drop(['AL000084', 'AL000204'])
len(clin_data_comput_ELBW)

#### F. Exclude one infant where detailed (manually collected) clinical data were not available 


In [ ]:
clin_data_comput_ELBW = clin_data_comput_ELBW.drop(['AT000265'])
len(clin_data_comput_ELBW)

#### G. Exclude infants who had <10 minutes of ventilator data available

In [ ]:
# Final set of included cases
# The cases included in `vent_modes` table have selected as having >10 minutes of ventilator data
cases = clin_data_comput_ELBW.index.intersection(vent_modes.index)
len(cases)

In [ ]:
# These are the babies where <10 minutes of ventilation data were not available
set(clin_data_comput_ELBW.index) - set(cases)

In [ ]:
# Limit clinical data and ventilation modes to the selected cases
clin_data_comput_ELBW = clin_data_comput_ELBW.loc[cases]
vent_modes_ELBW = vent_modes.loc[cases]
(len(clin_data_comput_ELBW), len(vent_modes_ELBW))

In [ ]:
# Mark computationally collected parameters with the tag `comput`
clin_data_comput_ELBW.rename(lambda x: f'{x}_comput', axis=1, inplace=True)

In [ ]:
clin_data_comput_ELBW.head(2)

In [ ]:
# Missing data means the ventilator mode did not occur in the recording
vent_modes_ELBW = vent_modes_ELBW.replace(np.nan, 0)
vent_modes_ELBW.head()

In [ ]:
# Limit blood gases to the final dataset
blood_gases_comput =  {recording : dta for recording, dta in blood_gases_comput.items() if recording in cases}
len(blood_gases_comput)

### 5. Export selected and processed clinical data, blood gases and ventilator modes to Excel files and pickle archives

In [ ]:
# Clinical data of final dataset
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'clin_data_comput_ELBW.xlsx')) as writer:
    clin_data_comput_ELBW.to_excel(writer, sheet_name='clin_data_comput')
with open(os.path.join(DATA_DUMP, 'clin_data_comput_ELBW.pickle'), 'wb') as handle:
    pickle.dump(clin_data_comput_ELBW, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Ventilation modes in the final dataset
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'ventilation_modes_ELBW.xlsx')) as writer:
    vent_modes_ELBW.to_excel(writer, sheet_name='ventilation modes')
with open(os.path.join(DATA_DUMP, 'ventilation_modes_ELBW.pickle'), 'wb') as handle:
    pickle.dump(vent_modes_ELBW, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Blood gases for the final dataset
with open(os.path.join(DATA_DUMP, 'blood_gases_comput.pickle'), 'wb') as handle:
    pickle.dump(blood_gases_comput, handle, protocol=pickle.HIGHEST_PROTOCOL)   

### 6. Import and process manually collected clinical data

#### A. Demographic and clinical data

In [ ]:
clin_data_manual = pd.read_excel(os.path.join(DIR_READ_CLIN, 'ELBW_table_v2_extended_1023.xlsx'),
                sheet_name='Demographic & clinical data' , header=0, index_col=[0,1])
clin_data_manual.reset_index(0, drop=True, inplace=True)
# Mark manually collected data with the tag `manual`
clin_data_manual.rename(lambda x: f'{x}_manual', axis=1, inplace=True)
len(clin_data_manual)

In [ ]:
column_names_orig = clin_data_manual.columns
column_names_orig;

In [ ]:
column_names_new = ['Date of Birth_manual', 'Arrival of PCAM_manual',
       'End of transport_manual', 'Km start_manual', 'Km end_manual', 'Km_manual', 'PCAM at birth_manual',
       'Born outside of hosp_manual', 'Gest Week_manual', 'Birth weight_manual', 'Sex_manual', 'Twins_manual',
       'First responder_manual', 'Mode of delivery_manual', 'Apgar 1_manual', 'Apgar 5_manual', 'Apgar 10_manual',
       'Antenatal steroid_manual', 'Maternal complications_manual', 'Delayed cord clamping_manual', 'Resp supp after birth_manual',
       'PIP_manual', 'PEEP_manual', 'Flow_manual', 'FiO2 (%)_manual', 'Who performed intub_manual',
       'Number of intubation attempts_manual', 'Size of ETT_manual', 'Chest compression_manual', 'Reintubation by PCAM_manual',
       'Surfactant admin_manual', 'Time of surfactant_manual', 'CV support drug_manual', 'Fluid bolus_manual', 'Adrenaline_manual',
       'Fentanyl_manual']

rename_dct = dict(zip(column_names_orig, column_names_new))
rename_dct;

In [ ]:
# Rename columns with simpler names
clin_data_manual.rename(rename_dct, axis=1, inplace=True)
clin_data_manual.info()

In [ ]:
clin_data_manual.head(2)

#### B. Snap II score

In [ ]:
snap = pd.read_excel(os.path.join(DIR_READ_CLIN, 'ELBW_table_v2_extended_1023.xlsx'), 
                        sheet_name='SNAPII-SNAPPE-II', header=0, index_col=[0,1])
# Mark columns with the tag `snap`
snap.reset_index(0, drop=True, inplace=True)
snap.rename(lambda x: f'{x}_snap', axis=1, inplace=True)

snap.info()

#### C. Outcome

In [ ]:
outcome = pd.read_excel(os.path.join(DIR_READ_CLIN, 'ELBW_table_v2_extended_1023.xlsx'), 
                    sheet_name='Outcome', header=0, index_col=[0,1])
outcome.reset_index(0, drop=True, inplace=True)
# Mark columns with the tag `outcome`
outcome.rename(lambda x: f'{x}_outcome', axis=1, inplace=True)

outcome.info()

#### D. Vital parameters

In [ ]:
vital_parameters = pd.read_excel(os.path.join(DIR_READ_CLIN, 'ELBW_table_v2_extended_1023.xlsx'), 
                          sheet_name='Vital parameters', header=[0,1], index_col=[0,1])
vital_parameters.reset_index(0, drop=True, inplace=True)
vital_parameters.columns.names = ['timing', 'parameter']
vital_parameters.info()

#### E. Blood gases

Blood gases at arrival of the transport service and at the end of transfer (during the handover to tertiary NICU). These data have been manually collected from clinical notes

In [ ]:
blood_gases_manual = pd.read_excel(os.path.join(DIR_READ_CLIN, 'ELBW_table_v2_extended_1023.xlsx'), 
                          sheet_name='Blood gas', header=[0,1], index_col=[0,1])
blood_gases_manual.reset_index(0, drop=True, inplace=True)
blood_gases_manual.columns.names = ['timing', 'parameter']
blood_gases_manual.info()

Blood gases were available in 49/55 and 50/50 cases, during arrival and at handover, respectively.
None of them was arterial

In [ ]:
# 1:capillary, 2:venous
blood_gases_manual['At arrival of PCAM']['type 1-capillary, 2-venous'].value_counts(dropna=False)

In [ ]:
# 1:capillary, 2:venous
blood_gases_manual['At arrival to NICU']['type 1-capillary, 2-venous'].value_counts(dropna=False)

In [ ]:
# Exclude venous blood gases
blood_gases_manual = blood_gases_manual.swaplevel(axis=1)
blood_gases_manual = blood_gases_manual[blood_gases_manual['type 1-capillary, 2-venous'] == 1]
blood_gases_manual = blood_gases_manual.swaplevel(axis=1)

### 7. Further process, combined and export the processed demographic, clinical and outcome data

In [ ]:
# Check that there is complete overlap between computationally identified and manually reviewed cases
(set(clin_data_comput_ELBW.index) - set(clin_data_manual.index), 
 set(clin_data_manual.index) - set(clin_data_comput_ELBW.index))

In [ ]:
clin_data_combined = pd.merge(clin_data_manual, clin_data_comput_ELBW, how='inner', left_index=True, right_index=True)
clin_data_combined = pd.merge(clin_data_combined, snap, how='left', left_index=True, right_index=True)
clin_data_combined = pd.merge(clin_data_combined, outcome, how='left', left_index=True, right_index=True)
clin_data_combined = clin_data_combined.sort_index(axis=1)

In [ ]:
# Add the date to the 'Time of surfactant_manual' column
# I have checked it manually that it never goes over to the next calendar day
clin_data_combined['Time of surfactant_manual'] = \
    clin_data_combined['Date of Birth_manual'].apply(lambda x:str(x)[:10]) + ' ' +\
    clin_data_combined['Time of surfactant_manual'].dropna().apply(str)

clin_data_combined['Time of surfactant_manual'] = pd.to_datetime(clin_data_combined['Time of surfactant_manual'])

In [ ]:
clin_data_combined.columns;

In [ ]:
# When there is multiple source of data, keep manually collected and verified data
columns_to_keep = ['Adrenaline_manual', 'Antenatal steroid_manual', 'Apgar 10_manual', 'Apgar 1_manual', 'Apgar 5_manual', 
                   'Arrival of PCAM_manual', 'BPD_outcome', 'Birth weight_manual', 'Born outside of hosp_manual',
                   'CV support drug_manual', 'Case ID_comput', 'Chest compression_manual', 'Date of Birth_manual', 'Death_outcome',
                   'Delayed cord clamping_manual', 'Duration_comput', 'End of transport_manual', 'Fentanyl_manual', 'FiO2 (%)_manual',
                   'FiO2_snap', 'First responder_manual', 'Flow_manual', 'Fluid bolus_manual', 'Gest Week_manual',
                   'ICD_comput', 'IVH_outcome', 'Inv. Vent (days)_outcome', 'Km end_manual', 'Km start_manual', 'Km_manual', 
                   'Length of AB (days)_outcome', 'Lowest pH_snap', 'Lowest temperature_snap', 'Maternal complications_manual', 
                   'Mean BP_snap', 'Mode of delivery_manual', 'Multiple seizures_snap', 'NEC_outcome', 'Neurodev. Impairment_outcome', 
                   'Non-inv. Vent (days)_outcome', 'Number of intubation attempts_manual', 'PCAM at birth_manual',
                   'PEEP_manual', 'PIP_manual', 'PO2/FiO2_snap', 'PO2_snap', 'Pathology_English_comput', 'Patient report_outcome',
                   'Postnatal Age_comput', 'ROP_outcome', 'Recording end_comput', 'Recording start_comput',
                   'Reintubation by PCAM_manual', 'Resp supp after birth_manual', 'SGA_snap', 'SNAP-II_snap', 'SNAPPE-II_snap', 
                   'Score.1_snap', 'Score.2_snap', 'Score.3_snap', 'Score.4_snap', 'Score.5_snap', 'Score.6_snap', 'Score_snap', 
                   'Sepsis_outcome', 'Sex_manual', 'Size of ETT_manual', 'Surfactant admin_manual',
                   'Time of surfactant_manual', 'Twins_manual', 'Urine output_snap', 'Who performed intub_manual']

clin_data_combined = clin_data_combined[columns_to_keep]

In [ ]:
clin_data_combined.info()

In [ ]:
clin_data_combined.head(2)

In [ ]:
# Export processed combined clinical dataset 
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'clin_data_combined.xlsx')) as writer:
    clin_data_combined.to_excel(writer, sheet_name='clin_data_combined')

In [ ]:
# Export processed combined clinical dataset as pickle archive
with open(os.path.join(DATA_DUMP, 'clin_data_combined.pickle'), 'wb') as handle:
    pickle.dump(clin_data_combined, handle, protocol=pickle.HIGHEST_PROTOCOL)   

### 8. Export vital parameters and blood gases at the arrival to the referring unit and and at the handover to tertiary NICU

In [ ]:
vital_parameters.head(2)

In [ ]:
blood_gases_manual.head(2)

In [ ]:
# Export processed table with vitals and gases 
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'vital_gases_manual.xlsx')) as writer:
    vital_parameters.to_excel(writer, sheet_name='vital_parameters_manual')
    blood_gases_manual.to_excel(writer, sheet_name='blood_gases_manual')

In [ ]:
# Vital parameters for the final dataset
with open(os.path.join(DATA_DUMP, 'vital_parameters_manual.pickle'), 'wb') as handle:
    pickle.dump(vital_parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)   

In [ ]:
# Blood gases for the final dataset
with open(os.path.join(DATA_DUMP, 'blood_gases_manual.pickle'), 'wb') as handle:
    pickle.dump(blood_gases_manual, handle, protocol=pickle.HIGHEST_PROTOCOL)   

### 9. Analyse clinical characteristics of cohort

#### A. Demographics

In [ ]:
clin_data_combined['Gest Week_manual'].describe()

In [ ]:
clin_data_combined.groupby('Gest Week_manual').size()

In [ ]:
clin_data_combined['Birth weight_manual'].describe()

In [ ]:
clin_data_combined.groupby('Gest Week_manual')['Birth weight_manual'].mean()

In [ ]:
# 1: male; 2: female
clin_data_combined['Sex_manual'].value_counts()

In [ ]:
# O: singleton; 1: twin
clin_data_combined['Twins_manual'].value_counts()

In [ ]:
# Export processed combined clinical dataset 
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'demographics_ELBW.xlsx')) as writer:
    clin_data_combined['Gest Week_manual'].describe().to_excel(writer, sheet_name='gest_age')
    clin_data_combined.groupby('Gest Week_manual').size().to_excel(writer, sheet_name='gest_by_week')
    clin_data_combined['Birth weight_manual'].describe().to_excel(writer, sheet_name='birth_weight')
    clin_data_combined['Sex_manual'].value_counts().to_excel(writer, sheet_name='male_1_female_2')
    clin_data_combined['Twins_manual'].value_counts().to_excel(writer, sheet_name='singleton_0_twin_1')

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))
ax.boxplot(clin_data_combined['Gest Week_manual'], 
    widths=0.3,  whis = [0, 100], patch_artist = True, showfliers = True, showmeans = True,
    medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
    capprops=capprops,)

ax.set_ylim(20, 32)
ax.set_xticks([1], ['Gestational age'])
ax.set_ylabel('Completed weeks')
fig.savefig(os.path.join (DIR_WRITE, f'gestational_age_ELBW_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))
ax.boxplot(clin_data_combined['Birth weight_manual'], 
    widths=0.3,  whis = [0, 100], patch_artist = True, showfliers = True, showmeans = True,
    medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
    capprops=capprops,)

ax.set_ylim(0, 1100)
ax.set_xticks([1], ['Birth weight'])
ax.set_ylabel('Grams')
fig.savefig(os.path.join (DIR_WRITE, f'birth_weight_ELBW_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

#### B. Antenatal care and delivery

##### 1. Antenatal steriods

0:no,
0.5:partial course,
1:full course

In [ ]:
clin_data_combined['Antenatal steroid_manual'].value_counts(dropna=False)

##### 2. Maternal complications

1:abruption, 2:infection, 3:cardiovascular problem, 4:umbilical cord problem, 5:other

In [ ]:
clin_data_combined['Maternal complications_manual'].value_counts(dropna=False)

##### 3. Born outside hospital

1:yes,
0:no

In [ ]:
clin_data_combined['Born outside of hosp_manual'].value_counts(dropna=False)

##### 4. Mode of delivery
1:vaginal,
2:caesarean section

In [ ]:
clin_data_combined['Mode of delivery_manual'].value_counts(dropna=False)

#### C. Delivery unit care

##### 1. Who was the first responder and arrival of Cerny

In [ ]:
# 1:parent/paramedic/other, 2:local paediatrician, 3:PCAM staff
clin_data_combined['First responder_manual'].value_counts(dropna=False)

In [ ]:
clin_data_combined['Age_at_Cerny_arrival'] = clin_data_combined['Arrival of PCAM_manual'] - \
                                             clin_data_combined['Date of Birth_manual']

In [ ]:
dpi=300; filetype = 'jpg'

fig,ax = plt.subplots(1,1, figsize=(12,5))
ax.plot(clin_data_combined['Age_at_Cerny_arrival'].sort_values()/ 1e9 / 60, marker = 'o', linestyle='')
ax.set_ylim(-180, 240)
plt.xticks(rotation = 90);
ax.axhline(y=0, color='black', linestyle='-')
ax.set_ylabel('Age at arrival of transport team [minutes]')

fig.savefig(os.path.join (DIR_WRITE, f'age_at_arrival_of transport_team.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=300; filetype = 'jpg'

fig,ax = plt.subplots(1,1, figsize=(12,5))
ax.plot(clin_data_combined['Age_at_Cerny_arrival'].sort_values()/ 1e9 / 60, color = 'black', 
        marker = 'o', linestyle='')
ax.set_ylim(-180, 240)
plt.xticks(rotation = 90);
ax.axhline(y=0, color='black', linestyle='-')
ax.set_ylabel('Age at arrival of transport team [minutes]')

fig.savefig(os.path.join (DIR_WRITE, f'age_at_arrival_of transport_team_bw.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
clin_data_combined['PCAM at birth'] = np.where(clin_data_combined['Age_at_Cerny_arrival']< pd.to_timedelta(0), 1, 0)

In [ ]:
# Cerny present at birth: 0:no, 1:yes
clin_data_combined['PCAM at birth'].value_counts()

In [ ]:
# Cerny present by 15 minutes of age
sum(clin_data_combined['Age_at_Cerny_arrival'] <= pd.to_timedelta('15 min'))

In [ ]:
# Cerny was not present at birth for those ones born outside hospital
clin_data_combined.groupby('Born outside of hosp_manual')['PCAM at birth'].value_counts()

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))

ax.boxplot(clin_data_combined['Age_at_Cerny_arrival']/ 1e9 / 60, widths=0.3, whis = [0, 100], 
           showfliers = False, showmeans = True,
           medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
           capprops=capprops, )
ax.set_ylim(-180, 240)
ax.set_xticks([1], ['Age at arrival\nof transport team'])
ax.set_ylabel('Minutes')
fig.savefig(os.path.join (DIR_WRITE, f'age_at_arrival_of transport_team_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

##### 2. Intubation and surfactant administration

In [ ]:
# 0-not intubated, 1-local pediatrician, 2-PCAM staff
clin_data_combined['Who performed intub_manual'].value_counts(dropna=False)

In [ ]:
# Cases who were not intubated
clin_data_combined[clin_data_combined['Who performed intub_manual'] == 0].index

In [ ]:
# 0: no, 1:yes
clin_data_combined['Reintubation by PCAM_manual'].value_counts(dropna=False)

In [ ]:
# Surfactant given prior to transport? 0: no, 1:yes
clin_data_combined['Surfactant admin_manual'].value_counts(dropna=False)

In [ ]:
clin_data_combined[clin_data_combined['Surfactant admin_manual'] == 0][['Gest Week_manual', 'Birth weight_manual',
                                                                        'Km_manual']]

In [ ]:
clin_data_combined['Age at surfactant'] = clin_data_combined['Time of surfactant_manual'] -\
clin_data_combined['Date of Birth_manual']

In [ ]:
clin_data_combined['Age at surfactant'].describe()

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))

ax.boxplot(clin_data_combined['Age at surfactant'].dropna() / 1e9 / 60, widths=0.3, whis = [0, 100], 
           showfliers=False, showmeans=True,
           medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
           capprops=capprops, )

ax.set_ylim(0, 240)
ax.set_xticks([1], ['Age at surfactant\nadministration'])
ax.set_ylabel('Minutes')
fig.savefig(os.path.join (DIR_WRITE, f'age_at_surfactant administration_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

##### 3. Chest compression and adrenaline bolus

In [ ]:
# 0:no; 1:yes
clin_data_combined['Chest compression_manual'].value_counts(dropna=False)

In [ ]:
# 0:no; 1:yes
clin_data_combined['Adrenaline_manual'].value_counts(dropna=False)

#### D. Transport characteristics

##### 1. Distance travelled

In [ ]:
clin_data_combined['Km_manual'].describe()

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))

ax.boxplot(clin_data_combined['Km_manual'], widths=0.3, whis = [0, 100], 
           showfliers = False, showmeans = True,
           medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
           capprops=capprops, )
ax.set_ylim(-5, 140)
ax.set_xticks([1], ['Transport distance'])
ax.set_ylabel('Kilometers')
fig.savefig(os.path.join (DIR_WRITE, f'transport distance_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

##### 2. Age at arrival

In [ ]:
clin_data_combined['Age at arrival_NICU'] = clin_data_combined['End of transport_manual'] - \
    clin_data_combined['Date of Birth_manual']

In [ ]:
clin_data_combined['Age at arrival_NICU'].describe()

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(3,4))

ax.boxplot(clin_data_combined['Age at arrival_NICU'].dropna() / 1e9 / 60, widths=0.3, whis = [0, 100], 
           showfliers=False, showmeans=True,
           medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
           capprops=capprops, )

ax.set_ylim(0, 400)
ax.set_xticks([1], ['Age at the end\nof transfer'])
ax.set_ylabel('Minutes')
fig.savefig(os.path.join (DIR_WRITE, f'age_at_arrival_NICU_boxplot.{filetype}'), 
            dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

#### E. Cardiovascular care and sedation during transport

In [ ]:
# 0:no; 1:yes
clin_data_combined['Fluid bolus_manual'].value_counts(dropna=False)

In [ ]:
# 0:no; 1:yes
clin_data_combined['CV support drug_manual'].value_counts(dropna=False)

In [ ]:
# 0:no; 1:yes
clin_data_combined['Fentanyl_manual'].value_counts(dropna=False)

#### F. Survival to discharge

In [ ]:
# 0: survived; 1: died
clin_data_combined['Death_outcome'].value_counts(dropna=False)

In [ ]:
((sum(clin_data_combined['Death_outcome']==0) / len(clin_data_combined)),
(sum(clin_data_combined['Death_outcome']==1) / len(clin_data_combined)))

In [ ]:
clin_data_combined.groupby('Gest Week_manual')['Death_outcome'].value_counts(dropna=False)

In [ ]:
# Export processed combined clinical dataset 
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'mortality_ELBW.xlsx')) as writer:
    clin_data_combined['Death_outcome'].value_counts(dropna=False).to_excel(writer, sheet_name='survived_0_died_1')
    clin_data_combined.groupby('Gest Week_manual')['Death_outcome'].value_counts(dropna=False).to_excel(writer, 
                                                                                        sheet_name='death_by_week')

##### Comparison with national statistics

In our cohort 30 / 55 survived
Nationally, inborn infants (2020) 288 / 374 survived

In [ ]:
stats.fisher_exact([[30, 55], [288, 374]])

#### G. Long-term morbidities

Morbidity data is only available for babies who survived to discharge

In [ ]:
clin_data_combined['Death_outcome'].value_counts(dropna=False)

In [ ]:
outcome[outcome['Death_outcome'] ==0].describe().loc['count']

In [ ]:
clin_data_combined[['Death_outcome', 'IVH_outcome']].value_counts(dropna=False)

In [ ]:
clin_data_combined[['Death_outcome', 'ROP_outcome']].value_counts(dropna=False)

In [ ]:
clin_data_combined[['Death_outcome', 'BPD_outcome']].value_counts(dropna=False)

In [ ]:
clin_data_combined[['Death_outcome', 'NEC_outcome']].value_counts(dropna=False)

In [ ]:
clin_data_combined['Inv. Vent (days)_outcome'].describe()

### 10. Compare selected parameters at arrival of the tranport service to referring hopital versus at handover to NICU

#### A. Body temperature

In [ ]:
temperature_arrival = vital_parameters['At arrival of PCAM']['Body temperature (°C)']
temperature_arrival.describe()

In [ ]:
temperature_handover = vital_parameters['At arrival to NICU']['Body temperature (°C)']
temperature_handover.describe()

In [ ]:
stats.mannwhitneyu(temperature_arrival.dropna(), temperature_handover.dropna())

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(4,4))

ax.boxplot([vital_parameters['At arrival of PCAM']['Body temperature (°C)'].dropna().values,
            vital_parameters['At arrival to NICU']['Body temperature (°C)'].dropna().values],
            widths=0.4, whis = [0, 100], 
            showfliers=False, showmeans=True,
            medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
            capprops=capprops, )

ax.set_ylim(27, 41)
ax.set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax.set_ylabel('Body temperature (°C)')

ax.text(1.5, 39, 'p=0.0002', horizontalalignment ='center')

fig.savefig(os.path.join(DIR_WRITE, f'Body_temperature_boxplot.{filetype}'), dpi = dpi, format = filetype,
           bbox_inches='tight', pad_inches=0.1);

#### B. Respiratory support

In [ ]:
vital_parameters['At arrival of PCAM']['Resp. support (1-mask, 2-cPAP, 3-intub)'].value_counts(dropna=False)

In [ ]:
vital_parameters['At arrival to NICU']['Resp. support (1-cPAP, 2-intub.)'].value_counts(dropna=False)

#### C. Blood gases

In [ ]:
blood_gases_arrival = blood_gases_manual['At arrival of PCAM'][['pH', 'pCO2', 'BE']]
blood_gases_arrival.describe()

In [ ]:
blood_gases_handover = blood_gases_manual['At arrival to NICU'][['pH', 'pCO2', 'BE']]
blood_gases_handover.describe()

In [ ]:
# Export processed combined clinical dataset 
with pd.ExcelWriter(os.path.join(DIR_WRITE, f'blood_gases_arrival_handover_ELBW.xlsx')) as writer:
    blood_gases_arrival.describe().to_excel(writer, sheet_name='arrival_of_team')
    blood_gases_handover.describe().to_excel(writer, sheet_name='handover_on_NICU')

##### pH

In [ ]:
stats.mannwhitneyu(blood_gases_arrival['pH'].dropna(), blood_gases_handover['pH'].dropna())

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(4,4))

ax.boxplot([blood_gases_arrival['pH'].dropna().values, blood_gases_handover['pH'].dropna().values],
            widths=0.4, whis = [5, 95], 
            showfliers=False, showmeans=True,
            medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
            capprops=capprops, )

ax.set_ylim(6.89, 7.5)
ax.set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax.set_ylabel('pH')

ax.text(1.5, 7.4, 'p<0.0001', horizontalalignment ='center')

fig.savefig(os.path.join(DIR_WRITE, f'pH_boxplot.{filetype}'), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

##### pCO2

In [ ]:
stats.mannwhitneyu(blood_gases_arrival['pCO2'].dropna(), blood_gases_handover['pCO2'].dropna())

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(4,4))

ax.boxplot([blood_gases_arrival['pCO2'].dropna(), blood_gases_handover['pCO2'].dropna()],
            widths=0.4, whis = [5, 95], 
            showfliers=False, showmeans=True,
            medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
            capprops=capprops, )

ax.set_ylim(21, 100)
ax.set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax.set_ylabel('pCO$_2$ [mmHg]')

ax.text(1.5, 90, 'p=0.01', horizontalalignment ='center')

fig.savefig(os.path.join(DIR_WRITE, f'pCO2_boxplot.{filetype}'), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

##### Base excess (BE)

In [ ]:
stats.mannwhitneyu(blood_gases_arrival['BE'].dropna(), blood_gases_handover['BE'].dropna())

In [ ]:
dpi=300; filetype = 'jpg'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,1, figsize=(4,4))

ax.boxplot([-1*blood_gases_arrival['BE'].dropna(), -1*blood_gases_handover['BE'].dropna()],
            widths=0.4, whis = [5, 95], 
            showfliers=False, showmeans=True,
            medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
            capprops=capprops, )

ax.set_ylim(-4, 20)
ax.set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax.set_ylabel('Base deficit [mmol/L]')

ax.text(1.5, 18, 'p=0.0001', horizontalalignment ='center')

fig.savefig(os.path.join(DIR_WRITE, f'BE_ELBW_boxplot.{filetype}'), dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

### 11. Produce the figures for the paper

#### A. Figure 1

In [ ]:
dpi=300; filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,axes = plt.subplots(2,2, figsize=(8,8))
fig.subplots_adjust(left=0.2, bottom=None, right=None, top=None, wspace=0.4, hspace=0.3)

# Figure 1A
axes[0,0].boxplot([vital_parameters['At arrival of PCAM']['Body temperature (°C)'].dropna().values,
    vital_parameters['At arrival to NICU']['Body temperature (°C)'].dropna().values],
    widths=0.4, whis = [0, 100], showfliers=False, showmeans=True, medianprops=medianprops, 
    meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )
axes[0,0].set_ylim(27, 41)
axes[0,0].set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
axes[0,0].set_ylabel('Body temperature (°C)')
axes[0,0].text(1.5, 39, 'p=0.0002', horizontalalignment ='center')

# Figure 1B
axes[0,1].boxplot(clin_data_combined['Age at surfactant'].dropna() / 1e9 / 60, widths=0.2, whis = [0, 100], showfliers=False, 
    showmeans=True, medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
    capprops=capprops, )
axes[0,1].set_ylim(0, 240)
axes[0,1].set_xticks([1], ['Age at surfactant\nadministration'])
axes[0,1].set_ylabel('Minutes')

# Figure 1C
axes[1,0].boxplot(clin_data_combined['Km_manual'], widths=0.2, whis = [0, 100], showfliers = False, showmeans = True,
    medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )
axes[1,0].set_ylim(-5, 140)
axes[1,0].set_xticks([1], ['Transport distance'])
axes[1,0].set_ylabel('Kilometers')

# Figure 1D
axes[1,1].boxplot(clin_data_combined['Age at arrival_NICU'].dropna() / 1e9 / 60, widths=0.2, whis = [0, 100], 
    showfliers=False, showmeans=True, medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, 
    whiskerprops=whiskerprops, capprops=capprops, )
axes[1,1].set_ylim(0, 400)
axes[1,1].set_xticks([1], ['Age at the end\nof transfer'])
axes[1,1].set_ylabel('Minutes')

fig.text(0.1, 0.93, 'A', fontsize=14); fig.text(0.54, 0.93, 'B', fontsize=14)
fig.text(0.1, 0.50, 'C', fontsize=14); fig.text(0.54, 0.50, 'D', fontsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'Figure_1.{filetype}'), dpi = dpi, format = filetype,
           bbox_inches='tight', pad_inches=0.1);

#### B. Figure 2

In [ ]:
dpi=300; filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}; whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}; flierprops = {'color': 'black', 'marker': '.'}

fig,ax = plt.subplots(1,3, figsize=(12,3))
fig.subplots_adjust(left=0.05, bottom=None, right=None, top=None, wspace=0.4, hspace=0.3)

# Figure 2A
ax[0].boxplot([blood_gases_arrival['pH'].dropna().values, blood_gases_handover['pH'].dropna().values], widths=0.4, whis = [5, 95], 
    showfliers=False, showmeans=True, medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, 
    whiskerprops=whiskerprops, capprops=capprops, )
ax[0].set_ylim(6.89, 7.5)
ax[0].set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax[0].set_ylabel('pH')
ax[0].text(1.5, 7.4, 'p<0.0001', horizontalalignment ='center')

# Figure 2B
ax[1].boxplot([blood_gases_arrival['pCO2'].dropna(), blood_gases_handover['pCO2'].dropna()], widths=0.4, whis = [5, 95], 
    showfliers=False, showmeans=True, medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, 
    whiskerprops=whiskerprops, capprops=capprops, )
ax[1].set_ylim(21, 100)
ax[1].set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax[1].set_ylabel('pCO$_2$ [mmHg]')
ax[1].text(1.5, 87, 'p=0.01', horizontalalignment ='center')

# Figure 2C
ax[2].boxplot([-1*blood_gases_arrival['BE'].dropna(), -1*blood_gases_handover['BE'].dropna()], widths=0.4, whis = [5, 95], 
    showfliers=False, showmeans=True, medianprops=medianprops, meanprops = meanpointprops, boxprops=boxprops, whiskerprops=whiskerprops, 
            capprops=capprops, )

ax[2].set_ylim(-4, 20)
ax[2].set_xticks([1,2], ['At arrival of\ntransport team', 'At the end of\ntransfer'])
ax[2].set_ylabel('Base deficit [mmol/L]')
ax[2].text(1.5, 16.5, 'p=0.0001', horizontalalignment ='center')

fig.text(0, 0.95, 'A', fontsize=14); fig.text(0.31, 0.95, 'B', fontsize=14); fig.text(0.63, 0.95, 'C', fontsize=14); 

fig.savefig(os.path.join(DIR_WRITE, f'Figure_2.{filetype}'), dpi = dpi, format = filetype,
    bbox_inches='tight', pad_inches=0.1);